**Задача о минимальном остовном дереве:**

* алгоритм Краскала на массиве
* алгоритм Краскала на дереве без рангов вершин (Kruskal's algorithm)

In [ ]:
#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import time
import random

In [ ]:
# создаем случайный неориентированный связный граф
def generate_random_graph(num_vertices):
    edges = []
    for u in range(num_vertices):                 # Перебираем все возможные ребра
        for v in range(u + 1, num_vertices):
          weight = random.randint(1, 100)          # Задаем вес каждому ребру от  1 до 10
          edges.append((u, v, weight))
    return edges


In [ ]:
num_vertices = 12  # Количество вершин
edges = generate_random_graph(num_vertices)
print(f"Random Edges:{edges}")

In [ ]:
#t1 = random_edges.copy()

### алгоритм Краскала на массиве

In [ ]:

def find(parent, v):
    # Поиск корня вершины
    while parent[v] != v:
        v = parent[v]
    return v

def union(parent, rank, u, v):
    # Объединение двух компонент
    root_u = find(parent, u)
    root_v = find(parent, v)

    if root_u != root_v:
        if rank[root_u] > rank[root_v]:
            parent[root_v] = root_u
        elif rank[root_u] < rank[root_v]:
            parent[root_u] = root_v
        else:
            parent[root_v] = root_u
            rank[root_u] += 1

def kruskal_mst(num_vertices, edges):
    # Сортируем рёбра по весу
    edges = sorted(edges, key=lambda x: x[2])

    parent = list(range(num_vertices))  # Каждая вершина изначально является своим корнем
    rank = [0] * num_vertices          # Ранг для объединения компонент
    result = []                        # Результирующий список рёбер MST

    e = 0  # Количество рёбер в MST
    i = 0  # Индекс для перебора рёбер

    # Пока не будет выбрано n-1 рёбер
    while e <= num_vertices - 1 and i < len(edges):
        u, v, weight = edges[i]
        i += 1
        x = find(parent, u)
        y = find(parent, v)

        # Если добавление ребра не создаёт цикл
        if x != y:
            e += 1
            result.append((u, v, weight))
            union(parent, rank, x, y)

    # Вывод результатов
    minimum_cost = sum(weight for _, _, weight in result)
    return result, minimum_cost

# Пример использования
if __name__ == "__main__":

    mst, total_cost = kruskal_mst(num_vertices, edges)
    # Создание графа для визуализации
    G = nx.Graph()
    G.add_weighted_edges_from(edges)

    # Создаём подграф MST для отображения минимального остовного дерева
    mst_graph = nx.Graph()
    mst_graph.add_weighted_edges_from(mst)

    # Позиции вершин
    pos = nx.spring_layout(G)

    # Визуализация всех рёбер
    nx.draw(
        G,
        pos,
        with_labels=True,
        edge_color="grey",
        width=0.5,
        node_size=500,
        font_size=10,
        alpha=0.8,
    )

    # Визуализация рёбер MST поверх основного графа
    nx.draw_networkx_edges(
        mst_graph,
        pos,
        edge_color="red",
        width=2,
    )
    plt.show()



In [ ]:
start_time = time.time()
kruskal_mst(num_vertices, edges)
print(f'Время работы {(time.time() - start_time) * 1000} ms')


mst, total_weight = kruskal_mst(num_vertices, edges)
print("Рёбра в минимальном остовном дереве:")
for u, v, weight in mst:
    print(f"{u} -- {v} == {weight}")

In [ ]:
iterations = 10
total_time = 0
for _ in range(iterations):
    start_time = time.time()
    kruskal_mst(num_vertices, edges)
    total_time += (time.time() - start_time)

print(f'Среднее время выполнения: {(total_time / iterations) * 1000:.3f} ms')

### алгоритм Краскала на дереве без рангов вершин

In [ ]:
# Преимущество: После первого вызова find, в будущем все вершины на пути к корню будут напрямую связаны с корнем, что значительно ускоряет последующие поиски.


def find(parent, v):
    # Поиск корня вершины с сжатием путей
    if parent[v] != v:
        parent[v] = find(parent, parent[v])  # Сжимаем путь
    return parent[v]

def union(parent, u, v):
    # Объединение двух компонент без рангов
    root_u = find(parent, u)
    root_v = find(parent, v)

    if root_u != root_v:
        parent[root_v] = root_u  # Просто объединяем компоненты

def kruskal_tree(num_vertices, edges):
    # Сортируем рёбра по весу
    edges = sorted(edges, key=lambda x: x[2])

    parent = list(range(num_vertices))  # Каждая вершина изначально является своим корнем
    result = []  # Результирующий список рёбер MST

    e = 0  # Количество рёбер в MST
    i = 0  # Индекс для перебора рёбер

    # Пока не будет выбрано n-1 рёбер
    while e < num_vertices - 1 and i < len(edges):
        u, v, weight = edges[i]
        i += 1

        root_u = find(parent, u)
        root_v = find(parent, v)

        # Если добавление ребра не создаёт цикл
        if root_u != root_v:
            e += 1
            result.append((u, v, weight))
            union(parent, u, v)

    # Вывод результатов
    minimum_cost = sum(weight for _, _, weight in result)
    return result, minimum_cost

# Пример использования
if __name__ == "__main__":
    mst, total_cost = kruskal_tree(num_vertices, edges)

    # Создание графа для визуализации
    G = nx.Graph()
    G.add_weighted_edges_from(edges)

    # Создаём подграф MST для отображения минимального остовного дерева
    mst_graph = nx.Graph()
    mst_graph.add_weighted_edges_from(mst)

    # Позиции вершин
    pos = nx.spring_layout(G, seed=42)

    # Визуализация всех рёбер
    nx.draw(
        G,
        pos,
        with_labels=True,
        edge_color="grey",
        width=0.5,
        node_size=500,
        font_size=10,
        alpha=0.8,
    )

    # Визуализация рёбер MST поверх основного графа
    nx.draw_networkx_edges(
        mst_graph,
        pos,
        edge_color="red",
        width=2,
    )
    plt.show()


 * Сортировка рёбер: O(E*logE)
 * Перебор рёбер и выполнение операций find и union: O(E*α(V)), где α — это обратная функция Аккермана, которая растёт очень медленно и для практических целей считается почти константной => O(E)

**Общая сложность алгоритма Краскала с использованием сжатия пути и без рангов — O(E*logE).**


In [ ]:
start_time = time.time()
kruskal_tree(num_vertices, edges)
print(f'Время работы {(time.time() - start_time) * 1000} ms')

mst_tree, total_weight = kruskal_tree(num_vertices, edges)
print("Edges in the Minimum Spanning Tree:")
for u, v, weight in mst_tree:
    print(f"{u} -- {v} == {weight}")

### Анализ производительности алгоритмов при увеличении графа

In [ ]:
vertices = []
time_ = []
time_tree = []
for i in range(10, 200, 10):
  vertices.append(i)
  random_edges = generate_random_graph(i) # Сздаем граф с разным кол-вом вершин

  start_time = time.time()
  kruskal_tree(num_vertices,edges)
  time_.append((time.time() - start_time) * 1000) # время выполнения на массиве и i вершинах

  start_time = time.time()
  kruskal_tree(i, random_edges)
  time_tree.append((time.time() - start_time) * 1000 )# время выполнения на дереве и i вершинах

In [ ]:
plt.plot(vertices, time_, label = 'на массиве')
plt.plot(vertices, time_tree, label = 'на дереве без рангов вершин')
plt.title ("Время работы алг. Краскала, (ms)")
plt.legend(loc='best', fontsize=12)
plt.show()

In [ ]:
# def generate_random_graph(num_vertices):
#     edges = []
#     connected = set()

#     # Создание остовного дерева
#     for u in range(1, num_vertices):
#         v = random.randint(0, u - 1)
#         weight = random.randint(1, 100)
#         edges.append((u, v, weight))
#         connected.add((min(u, v), max(u, v)))

#     # Добавление дополнительных рёбер
#     additional_edges = num_vertices * 2
#     for _ in range(additional_edges):
#         u, v = random.sample(range(num_vertices), 2)
#         edge = (min(u, v), max(u, v))
#         if edge not in connected:
#             weight = random.randint(1, 100)
#             edges.append((u, v, weight))
#             connected.add(edge)

#     return edges
